In [18]:
from itertools import product, combinations
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
#from numba import njit, jit
from scipy.linalg import expm
import sympy as sp
from sympy import Matrix 
import math
from sympy import simplify

In [2]:
# create computational basis for the system


def BasisCreator(arg1=None, arg2=None, arg3=None):
    basis_states = []
    
    if arg1 is not None and arg2 is None and arg3 is None:
        spin = list(product([1, -1], repeat=arg1))
        gauge = list(product([1, -1], repeat=arg1))
        return list(product(spin, gauge))
    
    if arg1 is not None and arg2 is not None and arg3 is None:
        lattice_configs = []
        gauge = list(product([1, -1], repeat=arg1))
        sample_list = list(range(arg1))
        for spin_configs in combinations(sample_list, arg2):
            lattice = [-1] * arg1
            for ind in spin_configs:
                lattice[ind] = 1
            lattice_configs.append(lattice)
        return list(product(lattice_configs, gauge))
    
    if arg1 is not None and arg2 is not None and arg3 is not None:
        lattice_configs = []
        gauge = list(product([1, -1], repeat=arg1))
        sample_list = list(range(arg1))
        for spin_configs in combinations(sample_list, arg2):
            lattice = [-1] * arg1
            for ind in spin_configs:
                lattice[ind] = 1
            lattice_configs.append(lattice)

        for site_config in lattice_configs:
            for gauge_field in gauge:
                valid = True
                for j in range(arg1):
                    tau_prev = gauge_field[j - 1 if j > 0 else arg1 - 1]
                    tau_next = gauge_field[j]
                    if -site_config[j] * tau_prev * tau_next != arg3[j]:
                        valid = False
                        break
                if valid:
                    basis_states.append((site_config, gauge_field))
        return basis_states
    
    return basis_states


In [3]:
#basis = BasisCreator(8,4,[1]*8)
basis = BasisCreator(4,2,[1]*4)
print("no. of basis states are {}  ".format( len(basis)))
bases = [(list(state[0]), list(state[1])) for state in basis]

no. of basis states are 12  


In [4]:
#define hamiltonian
def Ham(initial_stat, h ) :
    
    start_state = list(initial_stat).copy()
    site_0 = list(start_state[0])
    field_0 = list(start_state[1])

    for j in range(len(site_0)):
        ft = field_0[j]
        s = site_0[j]
        t_1 = j+1 if (j+1)< len(site_0) else 0
        s_1 = site_0[t_1]
        if s*s_1 == -1 :
                                     
            site_0[j] = -s
            site_0[t_1] = -s_1
            field_0[j] = -ft
        
    return site_0, field_0 
    
    

In [62]:
# create the hamiltonian matrix
def h_mat(h):
    new_state = []
    
    position = []
    tau_mat = []
    tau = int()
    ham_1 = np.zeros((len(basis), len(basis)))
    for i in range(len(bases)):
        n_site, n_field = Ham(bases[i], h)
        tau  = np.sum(n_field)
        tau2 = h*tau
        tau_mat.append(tau2)
        new_s = (n_site,n_field)
        new_state.append(new_s)
        try:
            index = bases.index(new_s)
            position.append([i,index])
        except ValueError:
            pass
    max_row = max(pair[1] for pair in position)
    max_col = max(pair[0] for pair in position)
    ham_2 = np.diag(tau_mat)
    for (j, i) in position:
        ham_1[i, j] = 1

    Ham_mat = ham_1 + ham_2
    ham_mat = sp.Matrix(Ham_mat)

    return Ham_mat

#_______________________

    


In [6]:

def evolstates(j_0, dt, h):
    rho0 = sp.eye(len(bases))
    n = len(bases)
    j = j_0
    psij = Matrix([1 if i == j else 0 for i in range(n)])
    hmat = h_mat(h)
    hmatnp = sp.Matrix(hmat)
    p_mat, d_mat = hmatnp.diagonalize()
    rho0_p = p_mat*rho0
    u1 = sp.exp(-1j*d_mat*dt) 
    u1d = sp.exp(1j*d_mat*dt)
    rhot = u1d * rho0_p * u1
    evs1 = rhot*psij
    state_t = sp.simplify(evs1)
    
    return state_t 

In [9]:
#def sigz(j_0, state_t):
    
sites, fields = extract_site_config_and_gauge_field(bases)
sig = []
sigi=int()
for i in range(len(sites)):
    st = sites[i]
    for t in range(len(sites[i])):
        
        sigi += (pow(-1,t))*st[t]/len(st)
    sig.append(sigi) 
    sigz = sp.Matrix(sig)
    

sigt = list(sigz)
sig_mat = np.diag(sigt)
sigz_t = sigz.transpose()
#sigmaz = state_t * sigz
 #   return sigmaz
    

In [60]:
hat4[0,4]

1.0

In [63]:
hat4 = h_mat(0)

In [64]:
sp.Matrix(hat4)

Matrix([
[  0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0],
[1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0],
[  0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0]])

In [72]:
basis

[([1, 1, -1, -1], (1, -1, -1, -1)),
 ([1, 1, -1, -1], (-1, 1, 1, 1)),
 ([1, -1, 1, -1], (1, 1, -1, -1)),
 ([1, -1, 1, -1], (-1, -1, 1, 1)),
 ([1, -1, -1, 1], (1, 1, 1, -1)),
 ([1, -1, -1, 1], (-1, -1, -1, 1)),
 ([-1, 1, 1, -1], (1, -1, 1, 1)),
 ([-1, 1, 1, -1], (-1, 1, -1, -1)),
 ([-1, 1, -1, 1], (1, -1, -1, 1)),
 ([-1, 1, -1, 1], (-1, 1, 1, -1)),
 ([-1, -1, 1, 1], (1, 1, -1, 1)),
 ([-1, -1, 1, 1], (-1, -1, 1, -1))]

In [77]:
st2 = Ham(bases[0], 0)

In [90]:
j =11
st2 = Ham(bases[j], 0)
print(st2 )
print(bases[j])
inx = bases.index(st2)
print(inx)

([-1, 1, 1, -1], [-1, 1, -1, -1])
([-1, -1, 1, 1], [-1, -1, 1, -1])
7


In [68]:
hat4sp = sp.Matrix(hat4)

In [71]:
d4

Matrix([
[-0.5 - 0.866025403784439*I,                          0,   0,                             0,                          0,                          0,                          0,                          0,   0,                             0,                          0,                          0],
[                         0, -0.5 + 0.866025403784439*I,   0,                             0,                          0,                          0,                          0,                          0,   0,                             0,                          0,                          0],
[                         0,                          0, 1.0,                             0,                          0,                          0,                          0,                          0,   0,                             0,                          0,                          0],
[                         0,                          0,   0, 1.0 + 1.64024101203819e-128*I,     

In [70]:
p4, d4 = hat4sp.diagonalize()

In [349]:
sigz_t * evs1

Matrix([[1.28108512330171 + 0.657557895682452*I]])

In [8]:
def extract_site_config_and_gauge_field(basis_states):
    site_configs = [state[0] for state in basis_states]
    gauge_fields = [state[1] for state in basis_states]
    return ( site_configs, gauge_fields)

In [347]:
evs0 = evolstates(5, 1, 1)
sp.simplify(sigz_t*evs1 )

Matrix([[1.28108512330171 + 0.657557895682452*I]])

In [632]:

sigt = list(sigz)
sig_mat = np.diag(sigt)
sig_mat= sp.Matrix(sig_mat)
sig_mat

Matrix([
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0, 1.0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0, 2.0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0, 2.0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0, 2.0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0, 2.0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0, 2.0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0, 1.0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0]])

In [633]:
rho0_p

Matrix([
[               1.0,  0.144098145370737, -0.228786537764133,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[ 0.144098145370737,   1.02076427549929, -0.326377458880999,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[-0.228786537764133, -0.326377458880999,    1.1384325572101,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[                 0,                  0,                  0,             

In [637]:
t =0

In [638]:
# diagonal ensemble

for i in range(len(bases)):
    t += sig_mat[i,i] * rho0_p[i,i]

t/12

1.12140576039126

In [583]:
def imbalance(j_0, t_f, dt, h):
    
    imbalance_array = []
    imbalance_value = complex()
    imb_val = float()
    n = int(t_f/dt)
    
    for t in range(n+1):
        
        evolution_state = (evolstates(j_0, t*dt, h))
        evstate = sp.Matrix(evolution_state)
        imbalance_value = (sigz_t*evstate)
        #imb_val = imbalance_value + imb_val
             
        imbalance_array.append(imbalance_value)
        
    return imbalance_array#, imb_val

In [515]:
sigz_t 

Matrix([[0, 0, 4, 8, 8, 8, 8, 8, 4, 0, 0, 0]])

In [561]:
imb_list = []
for i in range(len(simple_list)):
    imb_list_i= (simple_list[i].conjugate() +simple_list[i])/2
    imb_list.append(imb_list_i)

In [21]:
plt.plot(range(1000000), imb_list)

NameError: name 'imb_list' is not defined

In [494]:
simplify(ant**-1 * ant)

[[1.0]]

In [584]:
arr2, valu2 = imbalance (1, 1000, 1, 1)

ValueError: too many values to unpack (expected 2)

In [409]:
len(ar2)

101

array([[5.12676565961504 - 7.0797985229795*I]], dtype=object)

In [451]:
ar = imbalance(0, 100,1,1)

In [431]:
real_parts = [x.real for x in data]
imaginary_parts = [x.imag for x in data]

# Plotting the data
plt.figure(figsize=(10, 6))
plt.scatter(real_parts, imaginary_parts, color='blue', marker='o')

# Adding labels and title
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.title('Complex Numbers Visualization')
plt.axhline(0, color='grey', lw=0.5)
plt.axvline(0, color='grey', lw=0.5)
plt.grid(True)
plt.show()

AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [426]:
data = simple_list 

real_parts = [x.real for x in data]
imaginary_parts = [x.imag for x in data]

# Plotting the data
plt.figure(figsize=(10, 6))
plt.scatter(real_parts, imaginary_parts, color='blue', marker='o')

# Adding labels and title
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.title('Complex Numbers Visualization')
plt.axhline(0, color='grey', lw=0.5)
plt.axvline(0, color='grey', lw=0.5)
plt.grid(True)
plt.show()

AttributeError: 'Float' object has no attribute 'real'

In [425]:

real_parts1 = (np.real(x) for x in simple_list)
real_parts1

<generator object <genexpr> at 0x7f8b75ed7b50>

In [19]:
simple_list = [arr[0] for arr in ar]


NameError: name 'ar' is not defined

In [ ]:
from sympy import simplify
arr = []
# Example expression
for i in range(len(ar)):
    vi = simplify(ar[i])
    arr.append(vi)

In [20]:
rhot*psij

NameError: name 'rhot' is not defined

In [15]:
hat4 = sp.Matrix(hat4)

In [16]:
p_mat, d_mat = hat4.diagonalize()

In [17]:
d_mat

Matrix([
[2.11490754147676,                0,                  0,                0,                0,                  0,                 0,   0,                0,                                         0,                                      0,                                      0],
[               0, -1.8608058531117,                  0,                0,                0,                  0,                 0,   0,                0,                                         0,                                      0,                                      0],
[               0,                0, -0.254101688365052,                0,                0,                  0,                 0,   0,                0,                                         0,                                      0,                                      0],
[               0,                0,                  0, 2.11490754147676,                0,                  0,                 0,   0,                0,

In [616]:
# diagonal ensemble

for i in range(len(bases)):
    t += sig_mat[i,i] * rho0_p[i,i]

t/12

2.49281152078253

In [614]:
sig_mat 

Matrix([
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0, 1.0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0, 2.0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0, 2.0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0, 2.0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0, 2.0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0, 2.0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0, 1.0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0],
[0, 0,   0,   0,   0,   0,   0,   0,   0, 0, 0, 0]])

In [594]:
rho0_p = p_mat.conjugate().T @ rho0 @ p_mat
rho0_p=simplify(rho0_p)
rho0_p

Matrix([
[               1.0,  0.144098145370737, -0.228786537764133,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[ 0.144098145370737,   1.02076427549929, -0.326377458880999,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[-0.228786537764133, -0.326377458880999,    1.1384325572101,                   0,                   0,                  0,                  0,                 0,                  0,                                       0,                                       0,                                       0],
[                 0,                  0,                  0,             

In [586]:
p_mat

Matrix([
[ 0.991870569754787, 0.0309675777074431,  -0.254557520910085,                  0,                  0,                  0,                  0,                  0,                  0,                                         0,                                        0,                                        0],
[                 0,                  0,                   0,  0.991729081654829, 0.0300738498658941, -0.262159800019253,                  0,                  0,                  0,                                         0,                                        0,                                        0],
[                 0,                  0,                   0,   -0.1244581187988,  0.973540112166213,  0.064531400523403,                  0,                  0,                  0,                                         0,                                        0,                                        0],
[ 0.123506911195524,  0.979241617078082, -0.0646497798376285,

In [294]:
evs1 = sp.Matrix(evolstates(0,1,1))
row_matrix*evs1

Matrix([[1.28108512330171 + 0.657557895682452*I]])

In [293]:
row_matrix = sigz.transpose()
row_matrix

Matrix([[0, 0, 4, 8, 8, 8, 8, 8, 4, 0, 0, 0]])

In [218]:
evs1 = evolstates(1,1,1)
sp.simplify(evs1)

Matrix([
[ -0.185911659446584 - 0.263846332161939*I],
[-0.229047386690928 + 0.0547648653975879*I],
[ 0.101613599720544 - 0.0242956498725264*I],
[  -1.03649898136068 - 0.292784494231076*I],
[  0.645074736255775 + 0.629162741751528*I],
[-0.399893445708089 + 0.0956138860345638*I],
[                                        0],
[                                        0],
[                                        0],
[                                        0],
[                                        0],
[                                        0]])

In [215]:
evs1 = evolstates(0,1,1)
sp.simplify(evs1)

Matrix([
[  0.838021241587651 + 0.107568123354625*I],
[  0.129496261767532 + 0.355197173426773*I],
[  0.311332208007797 + 0.150941966763446*I],
[ 0.203654938328676 + 0.0261410790571546*I],
[0.0962949605761353 + 0.0123603885959484*I],
[  0.171023303492809 + 0.157219447867931*I],
[                                        0],
[                                        0],
[                                        0],
[                                        0],
[                                        0],
[                                        0]])

In [186]:
u1 = expm(-1j*matrixnp*1)
a =sp.Matrix(u1)
a

Matrix([
[  -0.353907578147117 - 0.800115064976731*I,                                          0,                                         0, -0.354015021182533 - 0.00687207187396061*I,   -0.676352198203329 - 0.468225208289862*I,                                          0,                                        0,                                         0,                                         0,                                        0,                                        0,                                         0],
[                                         0,    -0.480618954794061 + 1.01780919319103*I, -0.354015021182533 - 0.0068720718739608*I,                                          0,                                          0,    0.739707886526801 - 0.440736920794019*I,                                        0,                                         0,                                         0,                                        0,                                      

In [208]:
u1sp = sp.Matrix(u1)
rho0_p = up*rho0

In [209]:
rho0_p

Matrix([
[     -0.479099806058299 - 0.838493504882695*I,       0.118660301646703 - 0.0435939310612765*I,      -0.0277195777822624 + 0.540585784903084*I,  -2.03247022082966e-50 - 1.7009093550652e-48*I, -3.16199618458055e-50 - 6.37233555827901e-50*I, 2.60593364920168e-49 + 2.52488388227384e-50*I,                                             0,                                              0,                                             0,                                              0,                                              0,                                              0],
[9.22418639056953e-51 + 4.61093766273547e-50*I,   6.40211495519185e-49 - 3.9295160964851e-49*I, -1.46185847611737e-48 + 2.77260977425242e-49*I,     -0.0777772009991689 - 0.0680194632818472*I,      -0.0424016241261123 - 0.960644482040918*I,      0.0964293507022789 - 0.658083173359823*I,                                             0,                                              0,                                    

In [201]:
up, ud = u1sp.diagonalize()